In [1]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=9216dc4181666c825de590bee85205b7417a1c37c0cf80518ab4bfb3f8f8ba14
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [2]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install nest-asyncio
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 895.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255313 sha256=37846849869523ac1b069221baec87331bd5050e4895a746f9dbb7f35a4f4a27
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')
path ="/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from flask import Flask, request, jsonify
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import threading
import subprocess
import time
from pyngrok import ngrok

app = Flask(__name__)

# Charger le modèle SGDClassifier
sgd_classifier = joblib.load('/content/drive/MyDrive/models/final_model/model.pkl')

# Charger le MultiLabelBinarizer
multilabel_binarizer = joblib.load('/content/drive/MyDrive/multilabel_binarizer.pkl')

# Charger le TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/MyDrive/tfidf_vectorizer.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    text = data['text']

    # Vectorisation TF-IDF du texte
    text_tfidf = tfidf_vectorizer.transform([text])

    # Prédictions du modèle SGDClassifier
    predicted_labels = sgd_classifier.predict(text_tfidf)

    # Conversion des prédictions en tags prédits
    predicted_tags = multilabel_binarizer.inverse_transform(predicted_labels)

    # Nettoyage et formatage des tags prédits
    predicted_cleaned = [' '.join(pred).replace("[", "").replace("'", "").replace("\"", "").strip() for pred in predicted_tags]

    response = {
        'predicted_tags': predicted_cleaned[0]
    }

    return jsonify(response)

def run_flask_app():
    app.run(host='0.0.0.0', port=5011)  # Changer le port à 5001

# Lancer l'application Flask dans un thread séparé
flask_thread = threading.Thread(target=run_flask_app)
flask_thread.start()

# Attendre quelques secondes pour que l'application Flask démarre
time.sleep(5)

# Lancer ngrok dans un processus séparé
ngrok_process = subprocess.Popen(['ngrok', 'http', '5011'])  # Changer le port à 5001

# Attendre quelques secondes pour que ngrok démarre
time.sleep(5)

# Obtenir l'URL publique générée par ngrok
ngrok_info = subprocess.run(['curl', 'http://localhost:4040/api/tunnels'], capture_output=True, text=True)
public_url = ngrok_info.stdout.split('"public_url":"')[1].split('"')[0]
print("Public URL:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5011
 * Running on http://172.28.0.12:5011
INFO:werkzeug:Press CTRL+C to quit


Public URL: https://05d1-34-75-206-44.ngrok.io


In [27]:
import requests

api_url = public_url + '/predict'
data = {"text": " What is the best language for code c++ or javascript "}
headers = {"Content-Type": "application/json"}

response = requests.post(api_url, json=data, headers=headers)
print(response.json())


INFO:werkzeug:127.0.0.1 - - [30/Aug/2023 10:49:23] "POST /predict HTTP/1.1" 200 -


{'predicted_tags': ["['javascript',", "'language',"]}
